# Uso de RDD's en Spark

## Creación de un RDD 
Lo primero es obtener el objeto SparkContext. Recomendación personal: Hacerlo a partir del objeto SparkSession

In [ ]:
# IMPORTANTE:
# 1. Si se ejecuta en Cesga ignorar esta celda.

from pyspark.sql import SparkSession
# 2. Si se emplea el clúster adbgonzalez/spark-cluster:
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("01-rdd1") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "hdfs:///spark/logs/history") \
    .config("spark.history.fs.logDirectory", "hdfs:///spark/logs/history") \
    .getOrCreate()

spark.version  # Verifica la versión de Spark
# 3. Si se ejecuta emplando all-spark-notebook comentar  lo anterior y descomentar la siguiente línea:
#spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

### A partir de una colección local
Para crear un RDD a partir de una colección se usa el método parallelize. A continuación se muestran dos ejemplos: Uno para enteros y otro para strings

In [ ]:
rdd_int = sc.parallelize(range(10))
rdd_st = sc.parallelize ("Big Data aplicado. Curso de especialización de Inteligencia Artificial y Big Data".split())

También hay algún parámetro opcional, como el número de particiones.

In [ ]:
rdd_int_par = sc.parallelize(range(20),4)


rdd_st_par = sc.parallelize ("Big Data aplicado. Curso de especialización de Inteligencia Artificial y Big Data".split(),5)


### A partir de fuentes de datos
Para ello tenemos dos opciones:
- textfile: Para crear un RDD a partir de un archivo de texto
- wholeTextFiles: Para crear un RDD a partir de varios archivos de texto

In [ ]:
rdd_file = sc.textFile("hdfs:/user/jovyan/data/flight-data/csv/2015-summary.csv")
#rdd_whole_files = sc.wholeTextFiles("/home/jovyan/work/data/flight-data/csv")

### A partir de DataFrames existentes
Una forma muy sencilla de crear RDD's es a partir de un DataFrame o DataSet existente (se verán en próximas sesiones).

In [ ]:
spark.range(10).rdd.collect()

Así obtenemos un RDD formado por objetos de tipo *Row*. Para poder manejar estos datos, es necesario convertir estos objetos tipo *Row* al tipo correcto o extraer los valores, como se muestra en el sigiuente ejemplo:

In [ ]:
spark.range(10).toDF("id").rdd.map(lambda row: row[0]).collect()

## Acciones
A continuación vamos a ver algunas de las acciones más frecuentes:
### collect
Permite mostrar todos los elementos de un RDD


In [ ]:
print (rdd_int.collect())
print (rdd_st.collect())
print (rdd_file.collect())
#print (rdd_whole_files.collect())

### take
Permite obtener un número determinado de elmentos del RDD

In [ ]:
print (rdd_int.take(3))
print (rdd_st.take(3))
#print (rdd_file.take(2))
#print (rdd_whole_files.take(2))

### count
Devuelve el número de elementos de un RDD

In [ ]:
print (rdd_int.count())
print (rdd_st.count())
print (rdd_file.count())
print (rdd_whole_files.count())

In [ ]:
rdd_st.count()

### reduce
Permite, mediante una función especificada por el programador reducir el RDD a un único valor. Esta función recibe dos parámetros y devuelve un único resultado. Se pueden emplear también funciones *lambda*.


In [ ]:
# Sumamos todos los elementos
print(rdd_int.reduce (lambda x,y: x+y))

def word_length_reducer(word1,word2):
    if (len(word1) > len (word2)):
        return word1
    else:
        return word2


print (rdd_st.reduce (word_length_reducer))
print ( rdd_file.reduce (word_length_reducer)) # Cambiar?

### first
Devuelve el primer elemento de un RDD

In [ ]:
rdd_int.first()

In [ ]:
rdd_st.first()

### max/min
Devuelve el valor máximo/mínimo de un RDD



In [ ]:
rdd_int.min()

In [ ]:
rdd_st.max()

## Transformaciones
### map
Permite aplicar una función especificada por el programador a cada uno de los elementos del RDD devolviendo un RDD del mismo tamaño que el original

In [ ]:
rdd_int.map (lambda x: 2*x).collect()

In [ ]:
rdd_st.map (lambda x: list(x)).collect()

### flatMap
Permite realizar operaciones map que no sean 1:1. Por ejemplo, en el siguiente código aplicamos la misma función que en el caso anterior pero el resultado es muy distinto:


In [ ]:
rdd_st.flatMap(lambda x: list(x)).collect()

### distinct
Elimina los duplicados

In [ ]:
rdd_st.distinct().collect()

### filter
Permite seleccionar los elementos del RDD que cumplen determinada condición


In [ ]:
rdd_int.filter(lambda x: x % 2 == 0).collect()

In [ ]:
rdd_st.filter(lambda x: len(x) >= 5).collect()

### sortBy
Permite reordenar el RDD en función de un criterio que puede ser especificado mediante una función lambda. Si queremos hacer orden inverso tenemos que poner el valor en negativo.

In [ ]:
# Orden ascendente
rdd_st.sortBy(lambda x: len(x)).collect()

# Orden descendente
# rdd_st.sortBy(lambda x: len(x)*-1).collect()


### randomsplit
Permite dividir un RDD convirtiéndolo en un array de RDD’s en función de un array de pesos especificado por el programador

In [ ]:
for rdd in rdd_int.randomSplit([0.4, 0.6]):
    print(rdd.collect())

for rdd in rdd_st.randomSplit([0.5,0.5]):
    print(rdd.collect())


## Otras operaciones
### foreachPartition
Permite especificar que función aplicar a cada partición

In [ ]:
def f (iterator):
    for x in iterator:
        print(type(x))
        print (x)

rdd_int_par.pipe("wc -l").collect()
print(rdd_int_par.foreachPartition(f))


In [ ]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
partitions = rdd.glom().collect()  # glom() devuelve una lista de particiones
for i, partition in enumerate(partitions):
    print(f"Partition {i}: {partition}")


### glom
La función *glom* convierte cada partición del DataFrame en Arrays. Es una función que puede ser muy útil, pero si los arrays son muy grandes podría causar errores.

In [ ]:
rdd_st_par.glom().collect()

### Almacenar RDD's en archivos
Es posible almacenar los RDD's en archivos de texto. Dos métodos:
- **saveAsTextFile**: Almacena el RDD en archivos de texto. Hay que especificar la ruta y (opcionalmente) un códec de compresión:

In [ ]:
rdd_st.saveAsTextFile("/home/jovyan/texto.txt")

- **saveAsPickleFile**: En un entorno *HDFS* una *secuenceFile* es un archivo de texto formado por pares clave-valor binarios. Este método permite escribir un RDD como un *sequenceFile*:

In [ ]:
rdd_st.saveAsPickleFile("/home/jovyan/secuencia")

### Checkpointing
Permite almacenar estados intermedios para no tener que repetir la secuencia de operaciones desde el principio.

In [ ]:
sc.setCheckpointDir("/home/jovyan/checkpoints")
rdd_st.checkpoint()

### MapPartitions
El método **pipe** nos permite enviar, de forma similar a las tuberías de un shell, RDD's a la entrada de un comando del SO. A continuación se muestra un ejemplo sencillo con el comando **wc**:

In [ ]:
rdd_st_par.pipe("wc -l").collect()

Observando el código anterior destaca el hecho de que Spark ejecuta operaciones a nivel de partición. La función *map* es realmente un alias para ejecutar la función *mapPartitions* a nivel de fila. Este hecho hace posible ejecutar la operación *map* a nivel de fila, empleando la función *mapPartitions* e iteradores.

In [ ]:
cosa = rdd_st_par.mapPartitions(lambda x: [list(x)])
print(type(cosa))
cosa.collect()
